# 出租车GPS数据的读取与数据清洗

renhai: "这个文件主要是提取OD信息"

在这个教程中，你将会学到如何使用python的pandas包对出租车GPS数据进行数据清洗，识别出行OD

**原始数据来源：** https://people.cs.rutgers.edu/~dz220/data.html
**数据说明：** Taxi GPS Data Format: 22223,2013-10-22 08:49:25,114.116631,22.582466,0
Taxi ID, Time, Latitude, Longitude, Occupancy Status, Speed; Occupancy Status: 1-with passengers & 0-with passengers;
[Download](http://www-users.cs.umn.edu/~tianhe/BIGDATA/UrbanCPS/TaxiData/TaxiData)

<div class="alert alert-info"><h2>提供的基础数据是：</h2><p>    数据：<br>  
    1.出租车原始GPS数据(在data-sample文件夹下，原始数据集的抽样500辆车的数据)</p></div>

[pandas包的简介](https://baike.baidu.com/item/pandas/17209606?fr=aladdin)

## 读取数据

首先，读取出租车数据

In [1]:
import pandas as pd
#读取数据
data = pd.read_csv(r"Y:\hncdjzw\baidunetdiskdownload\城市数据发布V2\TaxiData",header = None)
data.head()

,0,1,2,3,4,5
0,22223,21:09:38,114.138535,22.609266,1,19
1,22223,11:14:18,114.137871,22.575317,0,0
2,22223,01:18:28,114.137131,22.575983,0,0
3,22223,13:11:42,114.136269,22.545851,1,18
4,22223,02:05:47,114.135948,22.578917,0,29


In [2]:
# 添加列标题 
data.columns = ['VehicleNum', 'Stime', 'Lng', 'Lat', 'OpenStatus', 'Speed']

In [3]:
#显示数据的前5行
data.head(5)

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed
0,22223,21:09:38,114.138535,22.609266,1,19
1,22223,11:14:18,114.137871,22.575317,0,0
2,22223,01:18:28,114.137131,22.575983,0,0
3,22223,13:11:42,114.136269,22.545851,1,18
4,22223,02:05:47,114.135948,22.578917,0,29


数据的格式：

VehicleNum —— 车牌  
Stime —— 时间  
Lng —— 经度  
Lat —— 纬度  
OpenStatus —— 是否有乘客(0没乘客，1有乘客)  
Speed —— 速度  

## 数据清洗

In [4]:
# 先检查有无空值
missing_values = data.isnull().sum()
missing_values

VehicleNum    0
Stime         0
Lng           0
Lat           0
OpenStatus    0
Speed         0
dtype: int64

In [5]:
# 将时间戳转换为时间格式
# 定义一个年月日字符串 不知道初始时则定义默认时间
default_date_str = '2013-10-22 '

# 将时间转换为字符串，并在前面加上默认日期
data['Stime'] = pd.to_datetime(default_date_str + data['Stime'])
data['Stime'] = pd.to_datetime(data['Stime'], format='%Y-%M-%d %H:%M:%S')
# 查看结果
data.head()

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed
0,22223,2013-10-22 21:09:38,114.138535,22.609266,1,19
1,22223,2013-10-22 11:14:18,114.137871,22.575317,0,0
2,22223,2013-10-22 01:18:28,114.137131,22.575983,0,0
3,22223,2013-10-22 13:11:42,114.136269,22.545851,1,18
4,22223,2013-10-22 02:05:47,114.135948,22.578917,0,29


我们将数据按车牌、时间排序，特别要注意，data排序后需要再赋值给data，否则没作用

In [6]:
# 将数据排序,并把排序后的数据赋值给原来的数据
data = data.sort_values(by = ['VehicleNum','Stime'])
data.head(5)

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed
157,22223,2013-10-22 00:00:00,114.145714,22.555317,1,80
1518,22223,2013-10-22 00:00:06,114.146950,22.555468,1,80
1145,22223,2013-10-22 00:00:14,114.148537,22.555866,1,80
1396,22223,2013-10-22 00:00:16,114.148949,22.555933,1,79
2165,22223,2013-10-22 00:00:22,114.150200,22.556116,1,84


我们现在要做的是，用出租车GPS数据识别OD：
数据按车牌、时间排序后，正常情况下的OpenStatus是这样的：

|OpenStatus |  |
| :-----------: |-----------|
|0||
|0||
|0||
|0||
|0||
|1|←此时乘客上车了|
|1||
|1||
|1||
|1||
|1||
|1||
|0|     ←此时乘客下车了|
|0||
|0||
|0||

但是，也会有时候有数据异常出现，比如：

|OpenStatus |  |
| ----------- |-----------|
|0||
|0||
|0||
|0||
|0||
|1|←异常|
|0||
|0||
|0||
|0||

或者

|OpenStatus |  |
| ----------- |-----------|
|1||
|1||
|1||
|1||
|1||
|0|←异常|
|1||
|1||
|1||
|1||

前后都是0，突然有一条数据变成1，或者前后都是1，突然变成0。这种异常情况我们是要排除的

在pandas的数据处理过程中，我们筛掉不要的数据用下面的方法是最好的

   > data[条件]是保留符合条件的数据  
    data[-(条件)]是删除符合条件的数据

    Series的shift()函数能够将数据按顺序后移一位
    Series的shift(-1)函数能够将数据按顺序前移一位
    因此我们要判断的是，如果：
    后一位和前一位相等，但是后一位与中间一位不等，那么中间一位的数据就要删除（前一条数据，中间一条数据，后一条数据的车牌必须相等）
    
所以，我们立马开始筛选，把异常数据删除：

In [27]:
#筛选前的数据量
len(data)

46927855

In [28]:
# 计算各个条件
# 检查当前行的下一行的‘openstatus’是否与前一行的‘openstatus’相同
same_open_status_next = data['OpenStatus'].shift(-1) == data['OpenStatus'].shift()

# 检查当前行的下一行的 'OpenStatus' 是否与当前行的 'OpenStatus' 不同
diff_open_status_current = data['OpenStatus'].shift(-1) != data['OpenStatus']

# 检查当前行的下一行的 'VehicleNum' 是否与前一行的 'VehicleNum' 相同
same_vehicle_num_next = data['VehicleNum'].shift(-1) == data['VehicleNum'].shift()

# 检查当前行的下一行的 'VehicleNum' 是否与当前行的前一行的 'VehicleNum' 相同
same_vehicle_num_previous = data['VehicleNum'].shift(-1) == data['VehicleNum'].shift()

# 将上述条件组合在一起，构成一个复合条件
# 这个复合条件检查是否 'OpenStatus' 在当前行与下一行相同但与前一行不同，
# 同时 'VehicleNum' 在当前行、前一行和下一行都相同
# 这样的行被视为不正常或异常的，需要从数据中过滤掉
condition = same_open_status_next & diff_open_status_current & same_vehicle_num_next & same_vehicle_num_previous

# 应用上述条件来过滤数据
# 使用 '~' 符号来选择不满足条件的行，即保留正常的数据行
filtered_data = data[~condition]

In [29]:
len(filtered_data)

46827682

## 识别出OD

## 乘客上下车的状态变化识别

In [30]:
data.head(5)

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed
157,22223,2013-10-22 00:00:00,114.145714,22.555317,1,80
1518,22223,2013-10-22 00:00:06,114.146950,22.555468,1,80
1145,22223,2013-10-22 00:00:14,114.148537,22.555866,1,80
1396,22223,2013-10-22 00:00:16,114.148949,22.555933,1,79
2165,22223,2013-10-22 00:00:22,114.150200,22.556116,1,84


接下来，我们把下一条数据的信息放到前一条数据上，这样子，就能很方便的比较这条数据和下条数据的差异

在字段名加个1，代表后面一条数据的值

另外我们定义StatusChange为下一条数据的OpenStatus减去这一条数据的OpenStatus，这样就会出现：




|OpenStatus     |   OpenStatus1    |  StatusChange||
| ----------- |-----------|||
|0          |       0    |             0||
|0          |       0    |             0||
|0         |        0    |             0||
|0          |       1    |             1 |    ←此时乘客上车了|
|1          |       1    |             0  |   ←此时乘客上车了|
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       1    |             0||
|1          |       0    |             -1|    ←此时乘客下车了|
|0          |       0    |             0  |  ←此时乘客下车了|
|0          |       0    |             0||
|0          |       0    |             0||
|0          |       0    |             0||

注意到，乘客上车和下车都有两条数据，一般我们认为这两条数据的位置和时间非常接近，都可以认为是下车或者上车的地点

In [31]:
# 列出需要进行 shift 操作的列名
columns_to_shift = ['OpenStatus', 'VehicleNum', 'Lng', 'Lat', 'Stime']

# 检查数据集是否至少有两行
if len(data) > 1:
    # 对每个列名进行操作
    for col in columns_to_shift:
        # 检查列是否存在
        if col in data.columns:
            # 创建新列，名字为原列名加1，值为下一行的值
            data[f'{col}1'] = data[col].shift(-1)
        else:
            print(f"警告：列 '{col}' 不存在于数据中。")

    # 计算 OpenStatus 的变化
    data['StatusChange'] = data['OpenStatus1'] - data['OpenStatus']
else:
    print("警告：数据集不足以进行 shift 操作。")


In [32]:
data.head(5)

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed,OpenStatus1,VehicleNum1,Lng1,Lat1,Stime1,StatusChange
157,22223,2013-10-22 00:00:00,114.145714,22.555317,1,80,1.0,22223.0,114.146950,22.555468,2013-10-22 00:00:06,0.0
1518,22223,2013-10-22 00:00:06,114.146950,22.555468,1,80,1.0,22223.0,114.148537,22.555866,2013-10-22 00:00:14,0.0
1145,22223,2013-10-22 00:00:14,114.148537,22.555866,1,80,1.0,22223.0,114.148949,22.555933,2013-10-22 00:00:16,0.0
1396,22223,2013-10-22 00:00:16,114.148949,22.555933,1,79,1.0,22223.0,114.150200,22.556116,2013-10-22 00:00:22,0.0
2165,22223,2013-10-22 00:00:22,114.150200,22.556116,1,84,1.0,22223.0,114.151817,22.556601,2013-10-22 00:00:30,0.0


In [33]:
data[data.StatusChange == 1]

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed,OpenStatus1,VehicleNum1,Lng1,Lat1,Stime1,StatusChange
1404,22223,2013-10-22 00:03:23,114.167465,22.562468,0,0,1.0,22223.0,114.167732,22.562550,2013-10-22 00:03:39,1.0
930,22223,2013-10-22 00:11:33,114.227150,22.554167,0,0,1.0,22223.0,114.227264,22.554234,2013-10-22 00:11:49,1.0
1975,22223,2013-10-22 00:17:13,114.231354,22.562166,0,0,1.0,22223.0,114.231598,22.562349,2013-10-22 00:17:29,1.0
1508,22223,2013-10-22 00:36:45,114.240196,22.563650,0,0,1.0,22223.0,114.239700,22.563450,2013-10-22 00:37:01,1.0
515,22223,2013-10-22 01:01:14,114.135414,22.575933,0,0,1.0,22223.0,114.136467,22.575583,2013-10-22 01:01:29,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
46921067,36947,2013-10-22 22:39:12,114.005547,22.548067,0,31,1.0,36947.0,114.006699,22.549067,2013-10-22 22:40:00,1.0
46921172,36947,2013-10-22 22:49:38,113.994598,22.535049,0,17,1.0,36947.0,113.990318,22.535851,2013-10-22 22:50:20,1.0
46922106,36947,2013-10-22 23:24:24,113.921082,22.513483,0,11,1.0,36947.0,113.922081,22.513317,2013-10-22 23:25:02,1.0
46921294,36947,2013-10-22 23:37:09,113.927635,22.512568,0,7,1.0,36947.0,113.927635,22.512568,2013-10-22 23:37:15,1.0


## 将上下车状态整理为OD

    这里，我只想保留StatusChange字段为1或者-1的数据，因此要把这些数据筛出来
    不过，还得加一个条件，就是这条数据和下一条数据的车辆ID必须是相同的

In [34]:
#1.StatusChange字段为1或者-1
#2.这条数据和下一条数据的车辆ID必须是相同的
# 条件1: StatusChange 字段的值为 1 或 -1
condition_status_change = (data['StatusChange'] == 1) | (data['StatusChange'] == -1)

# 条件2: 当前行和下一行的 VehicleNum 字段值相同
condition_vehicle_num_same = data['VehicleNum'] == data['VehicleNum1']

# 应用这两个条件过滤数据
data = data[condition_status_change & condition_vehicle_num_same]

# 显示前5行数据以检查结果
data.head(5)

,VehicleNum,Stime,Lng,Lat,OpenStatus,Speed,OpenStatus1,VehicleNum1,Lng1,Lat1,Stime1,StatusChange
1359,22223,2013-10-22 00:01:59,114.163780,22.562349,1,0,0.0,22223.0,114.163803,22.562349,2013-10-22 00:02:16,-1.0
1404,22223,2013-10-22 00:03:23,114.167465,22.562468,0,0,1.0,22223.0,114.167732,22.562550,2013-10-22 00:03:39,1.0
2008,22223,2013-10-22 00:10:48,114.225235,22.552750,1,0,0.0,22223.0,114.225487,22.552933,2013-10-22 00:11:04,-1.0
930,22223,2013-10-22 00:11:33,114.227150,22.554167,0,0,1.0,22223.0,114.227264,22.554234,2013-10-22 00:11:49,1.0
1158,22223,2013-10-22 00:15:19,114.229218,22.560217,1,0,0.0,22223.0,114.229202,22.560150,2013-10-22 00:15:35,-1.0


In [35]:
#data数据只保留一些我们需要的字段
data = data[['VehicleNum','Stime','Lng','Lat','StatusChange']]
data.head(5)

,VehicleNum,Stime,Lng,Lat,StatusChange
1359,22223,2013-10-22 00:01:59,114.163780,22.562349,-1.0
1404,22223,2013-10-22 00:03:23,114.167465,22.562468,1.0
2008,22223,2013-10-22 00:10:48,114.225235,22.552750,-1.0
930,22223,2013-10-22 00:11:33,114.227150,22.554167,1.0
1158,22223,2013-10-22 00:15:19,114.229218,22.560217,-1.0


    我们现在就得到了乘客哪里上车，哪里下车。
    而我们想要得到的OD数据形式是，每一行记录包括了信息：车辆ID，上车时间，上车地点，下车时间，下车地点
    这样一行数据就是一个OD
    所以，接下来的操作就是。。。

In [36]:
# 首先，确保数据按照 VehicleNum 和 Stime 排序
data.sort_values(by=['VehicleNum', 'Stime'], inplace=True)

In [37]:
data.head(5)

,VehicleNum,Stime,Lng,Lat,StatusChange
1359,22223,2013-10-22 00:01:59,114.163780,22.562349,-1.0
1404,22223,2013-10-22 00:03:23,114.167465,22.562468,1.0
2008,22223,2013-10-22 00:10:48,114.225235,22.552750,-1.0
930,22223,2013-10-22 00:11:33,114.227150,22.554167,1.0
1158,22223,2013-10-22 00:15:19,114.229218,22.560217,-1.0


In [38]:
# 筛选出statuschange为1的数据 代表乘客上车 

In [39]:
#  将S：start和E：end分别表示起点和终点的数据分开
data = data.rename(columns = {'Lng':'SLng','Lat':'SLat'})
data['ELng'] = data['SLng'].shift(-1)
data['ELat'] = data['SLat'].shift(-1)
data['Etime'] = data['Stime'].shift(-1)
data = data[data['StatusChange'] == 1]
data = data.drop('StatusChange',axis = 1)
###################################################################################

    大功告成！接下来就是保存  
    数据文件夹下有一个TaxiOD.csv文件是我用1.7GB的全部数据计算的OD

In [40]:
data.to_csv('./data-sample/TaxiOD-All.csv',index = None)